In [ ]:
# http://www.sefidian.com/2022/02/28/understanding-tf-idf-with-python-example/
# https://www.techiedelight.com/initialize-list-of-lists-python/
# https://pynative.com/python-file-open/
# http://www.pythonforbeginners.com/cheatsheet/python-file-handling
# https://www.geeksforgeeks.org/python-spilt-a-sentence-into-list-of-words/
# https://www.w3schools.com/python/python_ref_string.asp
# https://en.wikipedia.org/wiki/Tf%E2%80%93idf
# Python/Udemy_NLP_JPortilla/UPDATED_NLP_COURSE/03-Text-Classification/01-Feature-Extraction-from-Text.ipynb
# https://stackoverflow.com/questions/32768555/find-the-set-of-column-indices-for-non-zero-values-in-each-row-in-pandas-data-f

In [ ]:
# Objective: find commonest words that are unique to single doc

In [ ]:
# Error handling needs:
# #if len(docNames) =/= len(filenames)
# empty lists

In [ ]:
# 1.0 Imports
# 2.0 Functions
    # getText
# 3.0 Global variables
    # filepath
    # filenames
    # docName
    # stem.verbs = [verb_stems.txt1, verb_stems.txt2, etc.]    - document-specific unique list of text strings
    # nouns = [doc.1_nouns, doc.2_nouns, etc.]                 - list of nouns unique to each doc
    # adj = [doc1.adj, etc]
    # adv = [doc1.adv, etc]
    
# 4.0 set-up - populate docNames with text file contents
# 5.0 Clean docNames[files] ready for spacy, TF-IDF: -each txt file should have a vocab list of stem_verbs, nouns, adj, adv
    # make content lower case
    # remove stopwords and dropwords
    # remove punctuation
# 6.0 prep for POS extraction
    # spacy
#7.0 Gather bulk data
    # make a corpus from all 8 files
#8.0 All Text
    # make a word-set (vocab of unique words) of the corpus
    # manual Term Frequeny calc
    # manual inverse document frequency (IDF) calc
    # apply sklearn TfidfVectorizer
# 9.0 Nouns
    # make a corpus from all 8 files
    # make a word-set (vocab of unique words) of the corpus
    # manual Term Frequeny calc
    # manual inverse document frequency (IDF) calc
    # apply sklearn TfidfVectorizer
# 10.0 Verbs
    # make a corpus from all 8 files
    # make a word-set (vocab of unique words) of the corpus
    # manual Term Frequeny calc
    # manual inverse document frequency (IDF) calc
    # apply sklearn TfidfVectorizer
# 11.0 Adjs and Advs
    # make a corpus from all 8 files
    # make a word-set (vocab of unique words) of the corpus
    # manual Term Frequeny calc
    # manual inverse document frequency (IDF) calc
    # apply sklearn TfidfVectorizer


In [1]:
# 1.0 Imports
import string
import nltk
import re
import nltk.corpus
import pandas as pd
import numpy as np
import spacy

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ianda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# 2.0 Functions
    # getText
def getText(txtFileName):
    with open(txtFileName,'r') as txtfile:
        reader = txtfile.read()
    return reader

In [3]:
# 3.0 Global variables
    # filepath
filepath = 'C://Users//ianda//Dropbox (NSF CORE)//Coding Projects//Python//NSN_RevHerPodcast_TF-IDF//Test_Text_Files//'

    # filenames
filenames = ['Metamorphosis1.txt','Metamorphosis2b.txt','Metamorphosis3.txt','Metamorphosis4.txt','Metamorphosis5.txt',
             'Metamorphosis6.txt','Metamorphosis7.txt','Metamorphosis8.txt']

# set size of empty lists to number of filenames
list_len = len(filenames)

    # docName container for collection of lists of strings (strings = body copy of transcript)
#docs to be empty lists which will contain 1 string (the whole of one text file) - OG text

docNames = [[] for x in range(list_len)]

In [4]:
# 4.0 set-up - populate docNames with text file contents

# FIX THIS
#if len(docNames) == len(filenames):
    
#    Else
#End

for i in range(len(filenames)):
    fileName = filepath + filenames[i]
    docNames[i] = getText(fileName)

In [ ]:
print(docNames[5])

In [5]:
# 5.0 Clean docNames[files] ready for spacy, TF-IDF: -each txt file should have a vocab list of stem_verbs, nouns, adj, adv
    # make entire content lower case
all_lower = [[] for x in range(list_len)]

for i in range(len(filenames)):
    all_lower[i] = docNames[i].lower()

In [ ]:
print(all_lower[5])

In [6]:
    # 5.1 remove stopwords and dropwords DO BEFORE DEPUNC
stop = stopwords.words('english')
dropWords = ['oh','wow', 'so','goodness', 'yeah']

all_stopped = [[] for x in range(list_len)]

for i in range(len(filenames)):
        all_stopped[i] = " ".join([word for word in all_lower[i].split() if word not in (stop) and word not in (dropWords)])

In [ ]:
print(all_stopped[5])

In [7]:
    # 5.2 remove punctuation
# https://monkeylearn.com/blog/text-cleaning/
# punctuation removal attempt - DO AFTER DROP/STOP
all_depunc = [[] for x in range(list_len)]

for i in range(len(filenames)):
        all_depunc[i] = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", all_stopped[i])


In [ ]:
print(all_depunc[5])

In [8]:
# 6.0 prep for POS extraction
    # spacy
# make list of spacy tokens for each file
all_spacy_nlp = [[] for x in range(list_len)]

for i in range(len(filenames)):
    all_spacy_nlp[i] = nlp(all_depunc[i])

In [ ]:
print(all_spacy_nlp[5])

In [9]:
#7.0 Gather and sort the data
    # Create full text then separate out each POS

# verbs will need to be stemmed for proper comparison which means they will need an additional processing step that
# other POSs don't need

# Select all token.pos as verb for doc_verb_stem.list
all_verbs = [[] for x in range(list_len)]
all_nouns = [[] for x in range(list_len)]
all_adjs = [[] for x in range(list_len)]
all_advs = [[] for x in range(list_len)]

# need to figure out why this needs a double indent to work
for i in range(len(filenames)):
        all_verbs[i] = [token for token in all_spacy_nlp[i] if token.pos_ == 'VERB']
        all_nouns[i] = [str(token) for token in all_spacy_nlp[i] if token.pos_ == 'NOUN']
        all_adjs[i] = [str(token) for token in all_spacy_nlp[i] if token.pos_ == 'ADJ']
        all_advs[i] = [str(token) for token in all_spacy_nlp[i] if token.pos_ == 'ADV']

In [ ]:
print(all_nouns)

In [10]:
# 7.0 Gather bulk data
corpus_all = [string for string in all_depunc]

In [ ]:
print(corpus_all)

In [ ]:
len(corpus_all)

In [ ]:
type(all_depunc[5])

In [ ]:
type(corpus_all[5])

In [ ]:
# 8.0 All text
    # make a word-set (vocab of unique words) of the corpus
    # manual Term Frequeny calc
    # manual inverse document frequency (IDF) calc
    # apply sklearn TfidfVectorizer

In [11]:
# 8.1 create a word set/vocab for the corpus
all_words_set = set()
 
for doc in corpus_all:
    words_all = doc.split(' ')
    all_words_set = all_words_set.union(set(words_all))

In [12]:
print('Number of words in the corpus:',len(all_words_set))

Number of words in the corpus: 304


In [13]:
print('The words in the corpus: \n', all_words_set)

The words in the corpus: 
 {'side', 'work', 'incredible', 'behavior', 'detract', 'recognize', 'hard', 'with', 'traveled', 'metamorphosis', 'belong', 'dictation', 'imagine', 'saying', 'around', 'pushing', 'scenery', 'family', 'place', 'trick', 'further', 'vast', 'beautiful', 'you', 'up', 'thoughts', 'yeah', 'lived', 'outside', 'no', 'be', 'year', 'studies', 'still', 'today', 'versions', 'instructed', 'trusted', 'kind', 'challenging', 'several', 'surround', 'need', 'them', 'journey', 'journaling', 'multidimensional', 'memos', 'background', 'get', 'career', 'friends', 'slightly', 'want', 'simple', 'transformation', 'old', 'ways', 'authentic', 'check', 'tips', 'therapeutic', 'like', 'heart', 'placebo', 'from', 'say', 'write', 'attracted', 'husband', 'two', 'even', 'happen', 'ill', 'another', 'without', 'stream', 'weeks', 'sort', 'comfort', 'one', 'exceeding', 'three', 'fact', 'have', 'again', 'mentally', 'dimensions', 'unquote', 'study', 'feel', 'myself', 'self', 'tools', 'topic', 'however

In [14]:
# 8.2 manual calculation of Term Frequency

n_docs_all = len(corpus_all)         #·Number of documents in the corpus
n_words_set_all = len(all_words_set) #·Number of unique words in the
 
df_tf_all = pd.DataFrame(np.zeros((n_docs_all, n_words_set_all)), columns=all_words_set)
 
# Compute Term Frequency (TF)
for i in range(n_docs_all):
    words = corpus_all[i].split(' ') # Words in the document
    for w in words:
        df_tf_all[w][i] = df_tf_all[w][i] + (1 / len(words))

In [15]:
#This shows the frequency of each word in each document.
df_tf_all

,side,work,incredible,behavior,detract,recognize,hard,with,traveled,metamorphosis,...,needs,small,time,were,strategies,part,many,consciousness,evermore,pretty
0,0.029851,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.014925,...,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.025316,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,...,0.000000,0.012658,0.000000,0.012658,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.025316,0.012658,0.012658,0.012658,0.000000,0.012658,0.000000,0.000000,...,0.000000,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.021505,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,0.000000,0.010753,0.010753,0.000000
6,0.000000,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.025316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,...,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316


In [16]:
# 8.3 Manual calc of Inverse Document Frequency (IDF):

print("IDF of: ")
 
idf_all = {}
 
for w in all_words_set:
    k = 0    # number of documents in the corpus that contain this word
     
    for i in range(n_docs_all):
        if w in corpus_all[i].split():
            k += 1
             
    idf_all[w] =  np.log10(n_docs_all / k)
     
    print(f'{w:>15}: {idf_all[w]:>10}' )

IDF of: 
           side: 0.9030899869919435
           work: 0.4259687322722811
     incredible: 0.6020599913279624
       behavior: 0.9030899869919435
        detract: 0.9030899869919435
      recognize: 0.9030899869919435
           hard: 0.9030899869919435
           with: 0.6020599913279624
       traveled: 0.9030899869919435
  metamorphosis: 0.9030899869919435
         belong: 0.9030899869919435
      dictation: 0.9030899869919435
        imagine: 0.9030899869919435
         saying: 0.9030899869919435
         around: 0.9030899869919435
        pushing: 0.9030899869919435
        scenery: 0.9030899869919435
         family: 0.6020599913279624
          place: 0.9030899869919435
          trick: 0.9030899869919435
        further: 0.9030899869919435
           vast: 0.9030899869919435
      beautiful: 0.9030899869919435
            you: 0.6020599913279624
             up: 0.9030899869919435
       thoughts: 0.9030899869919435
           yeah: 0.6020599913279624
          lived: 0.

         nature: 0.9030899869919435
      challenge: 0.6020599913279624
       finished: 0.9030899869919435
          voice: 0.9030899869919435
          happy: 0.9030899869919435
         course: 0.9030899869919435
           been: 0.9030899869919435
           hold: 0.9030899869919435
        experts: 0.6020599913279624
          shift: 0.9030899869919435
         mental: 0.9030899869919435
            sad: 0.9030899869919435
           take: 0.9030899869919435
         moment: 0.6020599913279624
         really: 0.3010299956639812
         future: 0.6020599913279624
     struggling: 0.6020599913279624
           that: 0.9030899869919435
      grounding: 0.6020599913279624
          thats: 0.6020599913279624
         listen: 0.9030899869919435
            low: 0.9030899869919435
           tell: 0.6020599913279624
            see: 0.9030899869919435
          kinds: 0.6020599913279624
        changes: 0.4259687322722811
            key: 0.9030899869919435
         seeing: 0.903089986

In [17]:
# 8.3 ctd
df_tf_all_idf = df_tf_all.copy()
 
for w in all_words_set:
    for i in range(n_docs_all):
        df_tf_all_idf[w][i] = df_tf_all[w][i] * idf_all[w]

In [18]:
df_tf_all_idf

,side,work,incredible,behavior,detract,recognize,hard,with,traveled,metamorphosis,...,needs,small,time,were,strategies,part,many,consciousness,evermore,pretty
0,0.026958,0.006358,0.000000,0.000000,0.000000,0.000000,0.013479,0.000000,0.000000,0.013479,...,0.000000,0.000000,0.013479,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.043004,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.015242,0.000000,0.000000,0.000000,0.000000,0.007621,0.000000,0.000000,...,0.000000,0.005392,0.000000,0.011432,0.000000,0.000000,0.011432,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.015242,0.011432,0.011432,0.011432,0.000000,0.007621,0.000000,0.000000,...,0.000000,0.005392,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.009161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.009711,0.000000,0.009711,0.009711,0.000000
6,0.000000,0.005392,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023766,0.000000,...,0.023766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023766


In [19]:
# 8.4 apply sklearn TfidfVectorizer
tr_idf_all_model  = TfidfVectorizer()
tf_idf_all_vector = tr_idf_all_model.fit_transform(corpus_all)

In [20]:
print(type(tf_idf_all_vector), tf_idf_all_vector.shape)

<class 'scipy.sparse.csr.csr_matrix'> (8, 304)


In [21]:
all_words_set = tr_idf_all_model.get_feature_names()

In [22]:
print(all_words_set)

['20s', '80', 'absolutely', 'achieve', 'admit', 'again', 'almost', 'alone', 'along', 'although', 'another', 'anything', 'app', 'approach', 'areas', 'around', 'attracted', 'authentic', 'background', 'barriers', 'based', 'be', 'beautiful', 'been', 'beginning', 'behavior', 'beliefs', 'belong', 'best', 'biological', 'blood', 'boundaries', 'brain', 'broken', 'call', 'cant', 'capture', 'career', 'catch', 'challenge', 'challenging', 'change', 'changes', 'check', 'child', 'children', 'clearly', 'clinical', 'comfort', 'companies', 'confidence', 'consciousness', 'corny', 'could', 'course', 'crystallize', 'days', 'definitely', 'describe', 'despite', 'detract', 'detractors', 'dictation', 'different', 'dimensions', 'direction', 'disciplines', 'draw', 'early', 'effectively', 'emerging', 'employed', 'essentially', 'even', 'evermore', 'evolved', 'exceeding', 'experts', 'express', 'fact', 'family', 'feel', 'felt', 'finished', 'forest', 'form', 'fortunate', 'friends', 'from', 'fundamental', 'further', '

In [23]:
# 8.4
tf_idf_all_array = tf_idf_all_vector.toarray()

In [24]:
# 8.4
df_tf_all_idf = pd.DataFrame(tf_idf_all_array, columns = all_words_set)

In [25]:
df_tf_all_idf

,20s,80,absolutely,achieve,admit,again,almost,alone,along,although,...,would,write,wrong,yeah,year,years,yet,you,yourself,zone
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117758,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.117758,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.194513,0.000000,0.000000,0.000000,0.000000,0.000000,0.194513
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.312857,0.000000,0.000000,0.000000,0.000000,0.000000,0.262198,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.099813,0.000000,0.119097,0.000000,0.000000,0.000000,...,0.119097,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.117379,0.000000,0.098373,0.000000,0.000000,0.000000,0.234758,0.000000,...,0.000000,0.000000,0.000000,0.098373,0.000000,0.000000,0.098373,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.092585,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.092585,0.000000,0.000000,0.000000,0.000000,0.077593,0.092585,0.000000
6,0.113071,0.113071,0.000000,0.113071,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.113071,0.000000,0.000000,0.000000,0.000000,0.094762
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.175065,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.146718,0.000000,0.000000,0.000000


In [26]:
# 8.4.1 distill that out
cols = df_tf_all_idf.columns
bt = df_tf_all_idf.apply(lambda x: x > 0)
final_shape_all = [bt.apply(lambda x: list(cols[x.values]), axis=1)]

In [27]:
print(final_shape_all)

[0    [although, based, call, change, clearly, clini...
1    [approach, barriers, change, changes, comfort,...
2    [been, catch, confidence, happen, know, like, ...
3    [admit, almost, another, areas, authentic, bra...
4    [absolutely, admit, along, another, authentic,...
5    [again, anything, app, attracted, be, beliefs,...
6    [20s, 80, achieve, around, beginning, belong, ...
7    [alone, beautiful, children, corny, could, for...
dtype: object]


In [ ]:
# 9.0 Nouns
    # make a corpus from all 8 files
    # make a word-set (vocab of unique words) of the corpus
    # manual Term Frequeny calc
    # manual inverse document frequency (IDF) calc
    # apply sklearn TfidfVectorizer


In [28]:
print(all_nouns)

[['people', 'time', 'change', 'transformation', 'metamorphosis', 'call', 'future', 'right', 'past', 'self', 'self', 'versions', 'people', 'journey', 'kinds', 'years', 'side', 'practice', 'pharmaceutical', 'side', 'pharmaceutical', 'companies', 'work', 'technology', 'technology', 'startup', 'companies'], ['strategies', 'talk', 'minute', 'change', 'changes', 'zone', 'barriers'], ['write', 'confidence'], ['experts', 'motivation', 'tips', 'ways', 'number', 'friends', 'family', 'change', 'challenge', 'admit', 'regrets', 'shame', 'changes', 'future', 'days', 'areas', 'know', 'husband', 'patience', 'ones', 'areas', 'child', 'school', 'dimensions', 'kids', 'kinds', 'things', 'brain'], ['supporters', 'detractors', 'thing', 'recognize', 'today', 'sense', 'direction', 'growth', 'goals', 'wellness', 'background', 'behavior', 'change', 'experts', 'motivation', 'tips', 'ways', 'number', 'friends', 'family', 'change', 'challenge', 'admit', 'regrets', 'shame', 'changes', 'make'], ['form', 'school', 'p

In [29]:
print(all_nouns[5])

['form', 'school', 'paper', 'form', 'dictation', 'fact', 'part', 'app', 'sales', 'pitch', 'way', 'moment', 'paper', 'voice', 'memos', 'tools', 'stream', 'consciousness', 'thoughts', 'topic', 'thing', 'thing', 'challenging', 'thing', 'thing', 'reflect', 'crystallize', 'position', 'hold', 'beliefs', 'shift', 'career', 'work', 'home']


In [ ]:
len(all_nouns)

In [30]:
# 9.1 make a corpus from all 8 files

# https://www.programiz.com/python-programming/examples/flatten-nested-list

# how to put all_nouns into right format and pass through? It needs to be a list of strings not a list of lists. Inner 
# list must be joined.

corpus_nouns = [" ".join(lst) for lst in all_nouns]

In [31]:
print(corpus_nouns) # each sublist needs to be a string not a list, len should = 8

['people time change transformation metamorphosis call future right past self self versions people journey kinds years side practice pharmaceutical side pharmaceutical companies work technology technology startup companies', 'strategies talk minute change changes zone barriers', 'write confidence', 'experts motivation tips ways number friends family change challenge admit regrets shame changes future days areas know husband patience ones areas child school dimensions kids kinds things brain', 'supporters detractors thing recognize today sense direction growth goals wellness background behavior change experts motivation tips ways number friends family change challenge admit regrets shame changes make', 'form school paper form dictation fact part app sales pitch way moment paper voice memos tools stream consciousness thoughts topic thing thing challenging thing thing reflect crystallize position hold beliefs shift career work home', 'comfort zone boundaries steps goals boundaries things 

In [32]:
# 9.2 create a word set/vocab for the corpus
noun_words_set = set()
 
for doc in corpus_nouns:
    words_nouns = doc.split(' ')
    noun_words_set = noun_words_set.union(set(words_nouns))

In [33]:
print('Number of words in the corpus:',len(noun_words_set))

Number of words in the corpus: 115


In [34]:
print('The words in the corpus: \n', noun_words_set)

The words in the corpus: 
 {'side', 'work', 'moment', 'comfort', 'behavior', 'practice', 'recognize', 'future', 'fact', 'dimensions', 'metamorphosis', 'study', 'self', 'tools', 'dictation', 'topic', 'kinds', 'years', 'changes', 'zone', 'markers', 'beliefs', 'things', 'steps', 'mirror', 'companies', 'scenery', 'people', 'family', 'place', 'barriers', 'water', 'paper', 'pitch', 'thing', 'brain', 'thoughts', 'motivation', 'number', 'kids', 'reflect', 'crystallize', 'patience', 'change', 'detractors', 'year', 'goals', 'children', 'studies', 'areas', 'home', 'boundaries', 'minute', 'today', 'position', 'versions', 'startup', 'sense', 'globe', 'make', 'forest', '20s', 'challenging', 'blood', 'regrets', 'shame', 'talk', 'journey', 'call', 'pharmaceutical', 'memos', 'child', 'background', 'career', 'friends', 'form', 'transformation', 'know', 'ways', 'rates', 'direction', 'days', 'pressure', 'admit', 'wellness', 'right', 'tips', 'heart', 'confidence', 'way', 'men', 'placebo', 'past', 'write', 

In [35]:
# 9.3 manual Term Frequeny calc

n_docs_nouns = len(corpus_nouns)         #·Number of documents in the corpus
n_words_set_nouns = len(noun_words_set) #·Number of unique words in the
 
df_tf_nouns = pd.DataFrame(np.zeros((n_docs_nouns, n_words_set_nouns)), columns=noun_words_set)
 
# Compute Term Frequency (TF)
for i in range(n_docs_nouns):
    words = corpus_nouns[i].split(' ') # Words in the document
    for w in words:
        df_tf_nouns[w][i] = df_tf_nouns[w][i] + (1 / len(words))

In [36]:
# The dataframe above shows we have a column for each word and a row for each document.
#This shows the frequency of each word in each document.
df_tf_nouns

,side,work,moment,comfort,behavior,practice,recognize,future,fact,dimensions,...,voice,part,course,stream,consciousness,hold,experts,shift,weeks,technology
0,0.074074,0.037037,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074074
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.035714,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000
5,0.000000,0.029412,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,...,0.029412,0.029412,0.000000,0.029412,0.029412,0.029412,0.000000,0.029412,0.000000,0.000000
6,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
#    # 9.4 manual inverse document frequency (IDF) calc
print("IDF of: ")
 
idf_nouns = {}
 
for w in noun_words_set:
    k = 0    # number of documents in the corpus that contain this word
     
    for i in range(n_docs_nouns):
        if w in corpus_nouns[i].split():
            k += 1
             
    idf_nouns[w] =  np.log10(n_docs_nouns / k)
     
    print(f'{w:>15}: {idf_nouns[w]:>10}' )

IDF of: 
           side: 0.9030899869919435
           work: 0.6020599913279624
         moment: 0.6020599913279624
        comfort: 0.9030899869919435
       behavior: 0.9030899869919435
       practice: 0.9030899869919435
      recognize: 0.9030899869919435
         future: 0.6020599913279624
           fact: 0.9030899869919435
     dimensions: 0.9030899869919435
  metamorphosis: 0.9030899869919435
          study: 0.9030899869919435
           self: 0.9030899869919435
          tools: 0.9030899869919435
      dictation: 0.9030899869919435
          topic: 0.9030899869919435
          kinds: 0.6020599913279624
          years: 0.9030899869919435
        changes: 0.4259687322722811
           zone: 0.6020599913279624
        markers: 0.9030899869919435
        beliefs: 0.9030899869919435
         things: 0.6020599913279624
          steps: 0.9030899869919435
         mirror: 0.9030899869919435
      companies: 0.9030899869919435
        scenery: 0.9030899869919435
         people: 0.

In [38]:
# 9.4 ctd
df_tf_nouns_idf = df_tf_nouns.copy()
 
for w in noun_words_set:
    for i in range(n_docs_nouns):
        df_tf_nouns_idf[w][i] = df_tf_nouns[w][i] * idf_nouns[w]

In [39]:
df_tf_nouns_idf

,side,work,moment,comfort,behavior,practice,recognize,future,fact,dimensions,...,voice,part,course,stream,consciousness,hold,experts,shift,weeks,technology
0,0.066896,0.022299,0.000000,0.000000,0.000000,0.033448,0.000000,0.022299,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066896
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021502,0.000000,0.032253,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021502,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.033448,0.000000,0.033448,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022299,0.000000,0.000000,0.000000
5,0.000000,0.017708,0.017708,0.000000,0.000000,0.000000,0.000000,0.000000,0.026561,0.000000,...,0.026561,0.026561,0.000000,0.026561,0.026561,0.026561,0.000000,0.026561,0.000000,0.000000
6,0.000000,0.000000,0.026177,0.039265,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.039265,0.000000,0.000000,0.000000,0.000000,0.000000,0.039265,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
# 9.5 apply sklearn TfidfVectorizer

tr_idf_nouns_model  = TfidfVectorizer()
tf_idf_nouns_vector = tr_idf_nouns_model.fit_transform(corpus_nouns)

In [41]:
print(type(tf_idf_nouns_vector), tf_idf_nouns_vector.shape)

<class 'scipy.sparse.csr.csr_matrix'> (8, 115)


In [42]:
# 9.5 continued
noun_words_set = tr_idf_nouns_model.get_feature_names()

In [43]:
print(noun_words_set)

['20s', 'admit', 'app', 'areas', 'background', 'barriers', 'behavior', 'beliefs', 'blood', 'boundaries', 'brain', 'call', 'career', 'challenge', 'challenging', 'change', 'changes', 'child', 'children', 'comfort', 'companies', 'confidence', 'consciousness', 'course', 'crystallize', 'days', 'detractors', 'dictation', 'dimensions', 'direction', 'experts', 'fact', 'family', 'forest', 'form', 'friends', 'future', 'globe', 'goals', 'growth', 'heart', 'hold', 'home', 'husband', 'journey', 'kids', 'kinds', 'know', 'make', 'markers', 'memos', 'men', 'metamorphosis', 'minute', 'mirror', 'moment', 'motivation', 'nature', 'number', 'ones', 'paper', 'part', 'past', 'patience', 'people', 'pharmaceutical', 'pitch', 'place', 'placebo', 'position', 'practice', 'pressure', 'rates', 'recognize', 'reflect', 'regrets', 'right', 'sales', 'scenery', 'school', 'self', 'sense', 'shame', 'shift', 'side', 'startup', 'steps', 'strategies', 'stream', 'studies', 'study', 'supporters', 'talk', 'technology', 'thing',

In [44]:
# 9.5 continued
tf_idf_nouns_array = tf_idf_nouns_vector.toarray()

In [45]:
# 9.5 continued
df_tf_nouns_idf = pd.DataFrame(tf_idf_nouns_array, columns = noun_words_set)

In [46]:
df_tf_nouns_idf

,20s,admit,app,areas,background,barriers,behavior,beliefs,blood,boundaries,...,water,way,ways,weeks,wellness,work,write,year,years,zone
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.139066,0.000000,0.000000,0.165934,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.421546,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353288
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000
3,0.000000,0.171130,0.000000,0.408387,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.171130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.178344,0.000000,0.000000,0.212801,0.000000,0.212801,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.178344,0.000000,0.212801,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.150168,0.000000,0.000000,0.000000,0.000000,0.150168,0.000000,0.000000,...,0.000000,0.150168,0.000000,0.000000,0.000000,0.125852,0.000000,0.000000,0.000000,0.000000
6,0.206232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.206232,0.412464,...,0.000000,0.000000,0.000000,0.206232,0.000000,0.000000,0.000000,0.206232,0.000000,0.172839
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.348676,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [47]:
# 9.5.1 final shape
cols = df_tf_nouns_idf.columns
bt = df_tf_nouns_idf.apply(lambda x: x > 0)
final_shape_nouns = [bt.apply(lambda x: list(cols[x.values]), axis=1)]

In [48]:
print(final_shape_nouns) #might be better as dict

[0    [call, change, companies, future, journey, kin...
1    [barriers, change, changes, minute, strategies...
2                                  [confidence, write]
3    [admit, areas, brain, challenge, change, chang...
4    [admit, background, behavior, challenge, chang...
5    [app, beliefs, career, challenging, consciousn...
6    [20s, blood, boundaries, comfort, course, goal...
7    [children, forest, friends, globe, nature, one...
dtype: object]


In [49]:
# 10.0 Verbs
    # 10.1 make a corpus from all 8 files
    # 10.2 make a word-set (vocab of unique words) of the corpus
    # 10.3 manual Term Frequeny calc
    # 10.4 manual inverse document frequency (IDF) calc
    # 10.5 apply sklearn TfidfVectorizer


In [50]:
print(all_verbs) #list of list of spacy tokens

[[say, seeing, imagining, want, listen, think, know, went, disciplines, know, working, partner, ve, shifted, emerging, based], [employed, know, re, guess], [know, takes, catch, happen, s], [went, grounding, check, spoke, trusted, struggling, feel, know, felt, needed, make, felt, would, evolved, know, m, working, including, tell, working, know, get, draw, know, describe], [think, may, detract, makes, know, ca, grow, need, know, know, background, went, grounding, check, spoke, trusted, struggling, feel, know, felt, needed], [s, attracted, m, felt, express, work, capture, may, want, need, need, think, setting, parenting, setting], [keeps, stretching, feel, achieve, pushing, know, exceeding, say, take, keep, pushing, trick, told, tell, work, instructed, try, look, try, could, went, wake, beginning, took, resting], [ve, lived, traveled, needs, s, know, know, loved, could, know, spouse]]


In [51]:
len(all_verbs)

8

In [ ]:
print(all_verbs[5])

In [ ]:
len(all_verbs[5])

In [ ]:
type(all_verbs[5][1])

In [52]:
# 10.1.1 extract stems from verbs to normalise vs tense
stemmer = PorterStemmer()

all_verb_stems = []

for doc in all_verbs:
    for item in doc:
        all_verb_stems.append(stemmer.stem(str(item)))

In [ ]:
len(all_verbs)

In [53]:
type(all_verbs[5])

list

In [ ]:
type(all_verbs[5][1])

In [54]:
# 10.1.2
all_verbs_stemmerFeed = [[] for x in range(list_len)]

for i in range(len(all_verbs)):
    print(all_verbs[i])
    all_verbs_stemmerFeed[i] = [vs for vs in all_verbs[i]]

[say, seeing, imagining, want, listen, think, know, went, disciplines, know, working, partner, ve, shifted, emerging, based]
[employed, know, re, guess]
[know, takes, catch, happen, s]
[went, grounding, check, spoke, trusted, struggling, feel, know, felt, needed, make, felt, would, evolved, know, m, working, including, tell, working, know, get, draw, know, describe]
[think, may, detract, makes, know, ca, grow, need, know, know, background, went, grounding, check, spoke, trusted, struggling, feel, know, felt, needed]
[s, attracted, m, felt, express, work, capture, may, want, need, need, think, setting, parenting, setting]
[keeps, stretching, feel, achieve, pushing, know, exceeding, say, take, keep, pushing, trick, told, tell, work, instructed, try, look, try, could, went, wake, beginning, took, resting]
[ve, lived, traveled, needs, s, know, know, loved, could, know, spouse]


In [56]:
# 10.1.3
stemmer = PorterStemmer()
all_verbStems = [[] for x in range(list_len)]

for i in range(len(all_verbs)):
    all_verbStems[i] = [stemmer.stem(str(vs)) for vs in all_verbs_stemmerFeed[i]]

In [57]:
print(all_verb_stems) # WRONG - still needs to be subdivided into docs

['say', 'see', 'imagin', 'want', 'listen', 'think', 'know', 'went', 'disciplin', 'know', 'work', 'partner', 've', 'shift', 'emerg', 'base', 'employ', 'know', 're', 'guess', 'know', 'take', 'catch', 'happen', 's', 'went', 'ground', 'check', 'spoke', 'trust', 'struggl', 'feel', 'know', 'felt', 'need', 'make', 'felt', 'would', 'evolv', 'know', 'm', 'work', 'includ', 'tell', 'work', 'know', 'get', 'draw', 'know', 'describ', 'think', 'may', 'detract', 'make', 'know', 'ca', 'grow', 'need', 'know', 'know', 'background', 'went', 'ground', 'check', 'spoke', 'trust', 'struggl', 'feel', 'know', 'felt', 'need', 's', 'attract', 'm', 'felt', 'express', 'work', 'captur', 'may', 'want', 'need', 'need', 'think', 'set', 'parent', 'set', 'keep', 'stretch', 'feel', 'achiev', 'push', 'know', 'exceed', 'say', 'take', 'keep', 'push', 'trick', 'told', 'tell', 'work', 'instruct', 'tri', 'look', 'tri', 'could', 'went', 'wake', 'begin', 'took', 'rest', 've', 'live', 'travel', 'need', 's', 'know', 'know', 'love',

In [58]:
print(all_verbStems[5])

['s', 'attract', 'm', 'felt', 'express', 'work', 'captur', 'may', 'want', 'need', 'need', 'think', 'set', 'parent', 'set']


In [59]:
len(all_verbStems) #all_verbStems is the corpus for verbs

8

In [60]:
# 10.2 create a word set/vocab for the verbs' corpus
# Make list of unique vocab lists for (verb_stems, nouns, adjs, advs) - can be done with set() and .union - how?
unique_verbStems_list = []

for i in range(len(all_verbStems)):
    for item in all_verbStems[i]:
        if item in unique_verbStems_list:
            continue
        else:
            unique_verbStems_list.append(item)

In [61]:
print(unique_verbStems_list)

['say', 'see', 'imagin', 'want', 'listen', 'think', 'know', 'went', 'disciplin', 'work', 'partner', 've', 'shift', 'emerg', 'base', 'employ', 're', 'guess', 'take', 'catch', 'happen', 's', 'ground', 'check', 'spoke', 'trust', 'struggl', 'feel', 'felt', 'need', 'make', 'would', 'evolv', 'm', 'includ', 'tell', 'get', 'draw', 'describ', 'may', 'detract', 'ca', 'grow', 'background', 'attract', 'express', 'captur', 'set', 'parent', 'keep', 'stretch', 'achiev', 'push', 'exceed', 'trick', 'told', 'instruct', 'tri', 'look', 'could', 'wake', 'begin', 'took', 'rest', 'live', 'travel', 'love', 'spous']


In [62]:
# 10.2 continued - make list into set
verbStem_set = set(unique_verbStems_list)

In [63]:
print(verbStem_set)

{'work', 'base', 'employ', 'need', 'achiev', 'took', 'detract', 'attract', 'tri', 'trust', 'catch', 'ground', 'struggl', 'describ', 'stretch', 'begin', 'listen', 'get', 'tell', 'feel', 'background', 'want', 'draw', 'know', 'see', 'spoke', 'live', 'look', 'emerg', 'disciplin', 'guess', 'check', 're', 'captur', 'spous', 'went', 'evolv', 'trick', 'told', 'wake', 'love', 'imagin', 'rest', 'say', 'exceed', 'would', 'felt', 'may', 'm', 'could', 'express', 'instruct', 'grow', 'think', 'happen', 'parent', 'travel', 've', 'keep', 'push', 'set', 'make', 'shift', 'partner', 's', 'ca', 'take', 'includ'}


In [64]:
# corpus_nouns passed through this step without error so all_verbStems needs to be replaced by something which has the same form

In [65]:
print(corpus_nouns)

['people time change transformation metamorphosis call future right past self self versions people journey kinds years side practice pharmaceutical side pharmaceutical companies work technology technology startup companies', 'strategies talk minute change changes zone barriers', 'write confidence', 'experts motivation tips ways number friends family change challenge admit regrets shame changes future days areas know husband patience ones areas child school dimensions kids kinds things brain', 'supporters detractors thing recognize today sense direction growth goals wellness background behavior change experts motivation tips ways number friends family change challenge admit regrets shame changes make', 'form school paper form dictation fact part app sales pitch way moment paper voice memos tools stream consciousness thoughts topic thing thing challenging thing thing reflect crystallize position hold beliefs shift career work home', 'comfort zone boundaries steps goals boundaries things 

In [66]:
corpus_verbStems = [" ".join(lst) for lst in all_verbStems]

In [67]:
print(corpus_verbStems)

['say see imagin want listen think know went disciplin know work partner ve shift emerg base', 'employ know re guess', 'know take catch happen s', 'went ground check spoke trust struggl feel know felt need make felt would evolv know m work includ tell work know get draw know describ', 'think may detract make know ca grow need know know background went ground check spoke trust struggl feel know felt need', 's attract m felt express work captur may want need need think set parent set', 'keep stretch feel achiev push know exceed say take keep push trick told tell work instruct tri look tri could went wake begin took rest', 've live travel need s know know love could know spous']


In [68]:
# 10.3 manual Term Frequeny calc

n_docs_verbStems = len(all_verbStems)         #·Number of documents in the corpus
n_words_verbStem_set = len(verbStem_set) #·Number of unique words in the
 
df_tf_verbs = pd.DataFrame(np.zeros((n_docs_verbStems, n_words_verbStem_set)), columns=verbStem_set)
 
# Compute Term Frequency (TF)
for i in range(n_docs_verbStems):
    words = corpus_verbStems[i].split(' ') # Words in the document
    for w in words:
        df_tf_verbs[w][i] = df_tf_verbs[w][i] + (1 / len(words))

In [69]:
df_tf_verbs

,work,base,employ,need,achiev,took,detract,attract,tri,trust,...,keep,push,set,make,shift,partner,s,ca,take,includ
0,0.062500,0.0625,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.000000,0.0625,0.0625,0.000000,0.000000,0.00,0.00
1,0.000000,0.0000,0.25,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.00
2,0.000000,0.0000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.000000,0.0000,0.0000,0.200000,0.000000,0.20,0.00
3,0.080000,0.0000,0.00,0.040000,0.00,0.00,0.000000,0.000000,0.00,0.040000,...,0.00,0.00,0.000000,0.040000,0.0000,0.0000,0.000000,0.000000,0.00,0.04
4,0.000000,0.0000,0.00,0.095238,0.00,0.00,0.047619,0.000000,0.00,0.047619,...,0.00,0.00,0.000000,0.047619,0.0000,0.0000,0.000000,0.047619,0.00,0.00
5,0.066667,0.0000,0.00,0.133333,0.00,0.00,0.000000,0.066667,0.00,0.000000,...,0.00,0.00,0.133333,0.000000,0.0000,0.0000,0.066667,0.000000,0.00,0.00
6,0.040000,0.0000,0.00,0.000000,0.04,0.04,0.000000,0.000000,0.08,0.000000,...,0.08,0.08,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.04,0.00
7,0.000000,0.0000,0.00,0.090909,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.000000,0.0000,0.0000,0.090909,0.000000,0.00,0.00


In [70]:
# 10.4 manual inverse document frequency (IDF) calc

print("IDF of: ")
 
idf_verbs = {}
 
for w in verbStem_set:
    k = 0    # number of documents in the corpus that contain this word
     
    for i in range(n_docs_verbStems):
        if w in corpus_verbStems[i].split():
            k += 1
             
    idf_verbs[w] =  np.log10(n_docs_verbStems / k)
     
    print(f'{w:>15}: {idf_verbs[w]:>10}' )

IDF of: 
           work: 0.3010299956639812
           base: 0.9030899869919435
         employ: 0.9030899869919435
           need: 0.3010299956639812
         achiev: 0.9030899869919435
           took: 0.9030899869919435
        detract: 0.9030899869919435
        attract: 0.9030899869919435
            tri: 0.9030899869919435
          trust: 0.6020599913279624
          catch: 0.9030899869919435
         ground: 0.6020599913279624
        struggl: 0.6020599913279624
        describ: 0.9030899869919435
        stretch: 0.9030899869919435
          begin: 0.9030899869919435
         listen: 0.9030899869919435
            get: 0.9030899869919435
           tell: 0.6020599913279624
           feel: 0.4259687322722811
     background: 0.9030899869919435
           want: 0.6020599913279624
           draw: 0.9030899869919435
           know: 0.05799194697768673
            see: 0.9030899869919435
          spoke: 0.6020599913279624
           live: 0.9030899869919435
           look: 0

In [71]:
# 10.4 continued
df_tf_verbs_idf = df_tf_verbs.copy()
 
for w in verbStem_set:
    for i in range(n_docs_verbStems):
        df_tf_verbs_idf[w][i] = df_tf_verbs[w][i] * idf_verbs[w]

In [72]:
df_tf_verbs_idf

,work,base,employ,need,achiev,took,detract,attract,tri,trust,...,keep,push,set,make,shift,partner,s,ca,take,includ
0,0.018814,0.056443,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.056443,0.056443,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.225772,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.085194,0.000000,0.120412,0.000000
3,0.024082,0.000000,0.000000,0.012041,0.000000,0.000000,0.000000,0.000000,0.000000,0.024082,...,0.000000,0.000000,0.000000,0.024082,0.000000,0.000000,0.000000,0.000000,0.000000,0.036124
4,0.000000,0.000000,0.000000,0.028670,0.000000,0.000000,0.043004,0.000000,0.000000,0.028670,...,0.000000,0.000000,0.000000,0.028670,0.000000,0.000000,0.000000,0.043004,0.000000,0.000000
5,0.020069,0.000000,0.000000,0.040137,0.000000,0.000000,0.000000,0.060206,0.000000,0.000000,...,0.000000,0.000000,0.120412,0.000000,0.000000,0.000000,0.028398,0.000000,0.000000,0.000000
6,0.012041,0.000000,0.000000,0.000000,0.036124,0.036124,0.000000,0.000000,0.072247,0.000000,...,0.072247,0.072247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024082,0.000000
7,0.000000,0.000000,0.000000,0.027366,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038724,0.000000,0.000000,0.000000


In [73]:
# 10.5 apply sklearn TfidfVectorizer
tr_idf_verbs_model  = TfidfVectorizer()
tf_idf_verbs_vector = tr_idf_verbs_model.fit_transform(corpus_verbStems)

In [74]:
# 10.5 continued
verbStem_set = tr_idf_verbs_model.get_feature_names()

In [75]:
# 10.5 continued
tf_idf_verbs_array = tf_idf_verbs_vector.toarray()

In [76]:
# 10.5 continued
df_tf_verbs_idf = pd.DataFrame(tf_idf_verbs_array, columns = verbStem_set)

In [77]:
df_tf_verbs_idf

,achiev,attract,background,base,begin,ca,captur,catch,check,could,...,travel,tri,trick,trust,ve,wake,want,went,work,would
0,0.000000,0.000000,0.000000,0.285933,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.239634,0.000000,0.239634,0.181304,0.181304,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.587055,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.191603,0.000000,...,0.000000,0.000000,0.000000,0.191603,0.000000,0.000000,0.000000,0.144965,0.289929,0.228622
4,0.000000,0.000000,0.252505,0.000000,0.000000,0.252505,0.000000,0.000000,0.211619,0.000000,...,0.000000,0.000000,0.000000,0.211619,0.000000,0.000000,0.000000,0.160109,0.000000,0.000000
5,0.000000,0.283284,0.000000,0.000000,0.000000,0.000000,0.283284,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.237414,0.000000,0.179625,0.000000
6,0.191264,0.000000,0.000000,0.000000,0.191264,0.000000,0.000000,0.000000,0.000000,0.160294,...,0.000000,0.382528,0.191264,0.000000,0.000000,0.191264,0.000000,0.121277,0.121277,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.304000,...,0.362735,0.000000,0.000000,0.000000,0.304000,0.000000,0.000000,0.000000,0.000000,0.000000


In [78]:
#10.5.1 Final shape
cols = df_tf_verbs_idf.columns
bt = df_tf_verbs_idf.apply(lambda x: x > 0)
final_shape_verbs = [bt.apply(lambda x: list(cols[x.values]), axis=1)]

In [79]:
print(final_shape_verbs) #might be better as dict

[0    [base, disciplin, emerg, imagin, know, listen,...
1                            [employ, guess, know, re]
2                          [catch, happen, know, take]
3    [check, describ, draw, evolv, feel, felt, get,...
4    [background, ca, check, detract, feel, felt, g...
5    [attract, captur, express, felt, may, need, pa...
6    [achiev, begin, could, exceed, feel, instruct,...
7    [could, know, live, love, need, spous, travel,...
dtype: object]


In [80]:
# 11.0 Adjs and Advs
    # make a corpus from all 8 files
    # make a word-set (vocab of unique words) of the corpus
    # manual Term Frequeny calc
    # manual inverse document frequency (IDF) calc
    # apply sklearn TfidfVectorizer


In [81]:
# 11.1 make a corpus from all 8 files

# https://www.programiz.com/python-programming/examples/flatten-nested-list
# step 7
# how to put all_nouns into right format and pass through? It needs to be a list of strings not a list of lists. Inner 
# list must be joined.

# 11.1.2
    
# all_depunc is feed doc
# https://www.programiz.com/python-programming/examples/flatten-nested-list
corpus_adjs = [" ".join(lst) for lst in all_adjs]
corpus_advs = [" ".join(lst) for lst in all_advs]

In [82]:
print(corpus_adjs) #print(corpus_adjs[2]) is empty. Let's foce it for now

['hard potential different present finished several different clinical therapeutic smaller', 'simple ill multidimensional fundamental', '', 'authentic small incredible incredible vulnerable vulnerable overwhelmed scared early many different', 'important much important own mental authentic small incredible incredible vulnerable vulnerable overwhelmed', 'old evermore great happy sad nervous unquote wrong', 'small small great clinical old imagine much much biological', 'little much simple vast untouched low key quiet beautiful']


In [83]:
print(corpus_adjs[2])

In [84]:
corpus_adjs[2]="replacementword"

In [85]:
print(corpus_adjs)

['hard potential different present finished several different clinical therapeutic smaller', 'simple ill multidimensional fundamental', 'replacementword', 'authentic small incredible incredible vulnerable vulnerable overwhelmed scared early many different', 'important much important own mental authentic small incredible incredible vulnerable vulnerable overwhelmed', 'old evermore great happy sad nervous unquote wrong', 'small small great clinical old imagine much much biological', 'little much simple vast untouched low key quiet beautiful']


In [86]:
# 11.2 create a word set/vocab for the verbs' corpus
adj_words_set = set()
adv_words_set = set()
 
for doc in corpus_adjs:
    words_adjs = doc.split(' ')
    adj_words_set = adj_words_set.union(set(words_adjs))
    
for doc in corpus_advs:
    words_advs = doc.split(' ')
    adv_words_set = adv_words_set.union(set(words_advs))


In [87]:
#11.3 manual Term Frequeny calc
            #11.3.1 adjectives

n_docs_adjs = len(corpus_adjs)         #·Number of documents in the corpus
n_words_set_adjs = len(adj_words_set) #·Number of unique words in the
 
df_tf_adjs = pd.DataFrame(np.zeros((n_docs_adjs, n_words_set_adjs)), columns=adj_words_set)
 
# Compute Term Frequency (TF)
for i in range(n_docs_adjs):
    words = corpus_adjs[i].split(' ') # Words in the document
    for w in words:
        df_tf_adjs[w][i] = df_tf_adjs[w][i] + (1 / len(words))

In [88]:
#11.3.2 adverbs

n_docs_advs = len(corpus_advs)         #·Number of documents in the corpus
n_words_set_advs = len(adv_words_set) #·Number of unique words in the
 
df_tf_advs = pd.DataFrame(np.zeros((n_docs_advs, n_words_set_advs)), columns=adv_words_set)
 
# Compute Term Frequency (TF)
for i in range(n_docs_advs):
    words = corpus_advs[i].split(' ') # Words in the document
    for w in words:
        df_tf_advs[w][i] = df_tf_advs[w][i] + (1 / len(words))

In [89]:
# 11.4 manual inverse document frequency (IDF) calc
    #11.4.1 adjectives
print("IDF of: ")
 
idf_adjs = {}
 
for w in adj_words_set:
    k = 0    # number of documents in the corpus that contain this word
     
    for i in range(n_docs_adjs):
        if w in corpus_adjs[i].split():
            k += 1
             
    idf_adjs[w] =  np.log10(n_docs_adjs / k)
     
    print(f'{w:>15}: {idf_adjs[w]:>10}' )

IDF of: 
     incredible: 0.6020599913279624
          early: 0.9030899869919435
    overwhelmed: 0.6020599913279624
           hard: 0.9030899869919435
multidimensional: 0.9030899869919435
           much: 0.4259687322722811
        unquote: 0.9030899869919435
            low: 0.9030899869919435
            old: 0.6020599913279624
         simple: 0.6020599913279624
       clinical: 0.6020599913279624
      authentic: 0.6020599913279624
        imagine: 0.9030899869919435
     biological: 0.9030899869919435
     vulnerable: 0.6020599913279624
            key: 0.9030899869919435
          quiet: 0.9030899869919435
      important: 0.9030899869919435
    therapeutic: 0.9030899869919435
      potential: 0.9030899869919435
replacementword: 0.9030899869919435
            own: 0.9030899869919435
           vast: 0.9030899869919435
      beautiful: 0.9030899869919435
         little: 0.9030899869919435
          wrong: 0.9030899869919435
        present: 0.9030899869919435
      different: 0

In [90]:
# 11.4 continued
       # 11.4.1 Adjectives
df_tf_adjs_idf = df_tf_adjs.copy()
 
for w in adj_words_set:
    for i in range(n_docs_adjs):
        df_tf_adjs_idf[w][i] = df_tf_adjs[w][i] * idf_adjs[w]

In [91]:
df_tf_adjs_idf

,incredible,early,overwhelmed,hard,multidimensional,much,unquote,low,old,simple,...,great,ill,many,happy,untouched,mental,sad,evermore,nervous,several
0,0.000000,0.000000,0.000000,0.090309,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090309
1,0.000000,0.000000,0.000000,0.000000,0.225772,0.000000,0.000000,0.000000,0.000000,0.150515,...,0.000000,0.225772,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.109465,0.082099,0.054733,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.082099,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.100343,0.000000,0.050172,0.000000,0.000000,0.035497,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.075257,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.112886,0.000000,0.075257,0.000000,...,0.075257,0.000000,0.000000,0.112886,0.000000,0.000000,0.112886,0.112886,0.112886,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.094660,0.000000,0.000000,0.066896,0.000000,...,0.066896,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.047330,0.000000,0.100343,0.000000,0.066896,...,0.000000,0.000000,0.000000,0.000000,0.100343,0.000000,0.000000,0.000000,0.000000,0.000000


In [92]:
#11.4.2 adverbs

print("IDF of: ")
 
idf_advs = {}
 
for w in adv_words_set:
    k = 0    # number of documents in the corpus that contain this word
     
    for i in range(n_docs_advs):
        if w in corpus_advs[i].split():
            k += 1
             
    idf_advs[w] =  np.log10(n_docs_advs / k)
     
    print(f'{w:>15}: {idf_advs[w]:>10}' )

IDF of: 
    essentially: 0.9030899869919435
           sort: 0.9030899869919435
      knowingly: 0.9030899869919435
    effectively: 0.9030899869919435
        further: 0.9030899869919435
        quickly: 0.9030899869919435
         really: 0.3010299956639812
           well: 0.6020599913279624
         almost: 0.9030899869919435
          again: 0.9030899869919435
       mentally: 0.9030899869919435
        clearly: 0.9030899869919435
     definitely: 0.9030899869919435
       slightly: 0.9030899869919435
     absolutely: 0.9030899869919435
       somewhat: 0.9030899869919435
          alone: 0.9030899869919435
           even: 0.9030899869919435
          still: 0.2041199826559248
      privately: 0.9030899869919435
        however: 0.9030899869919435
            yet: 0.9030899869919435
         nearby: 0.9030899869919435
           kind: 0.4259687322722811
             so: 0.9030899869919435
          there: 0.9030899869919435
          right: 0.6020599913279624
      overnight: 0.

In [93]:
# 11.4.2 Adverbs
df_tf_advs_idf = df_tf_advs.copy()
 
for w in adv_words_set:
    for i in range(n_docs_advs):
        df_tf_advs_idf[w][i] = df_tf_advs[w][i] * idf_advs[w]

In [94]:
df_tf_advs_idf

,essentially,sort,knowingly,effectively,further,quickly,really,well,almost,again,...,nearby,kind,so,there,right,overnight,best,fortunate,real,pretty
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043004,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.086009,0.000000,0.000000,0.000000,0.129013,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.425969,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.451545,0.000000,0.000000,0.000000,0.000000
3,0.100343,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066896,0.100343,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.133791,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.112886,0.000000,0.000000,0.075257,0.075257,0.000000,0.000000,...,0.000000,0.000000,0.112886,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.090309,0.060206,0.000000,0.000000,0.090309,...,0.000000,0.085194,0.000000,0.090309,0.000000,0.000000,0.090309,0.000000,0.000000,0.000000
6,0.000000,0.129013,0.129013,0.000000,0.129013,0.000000,0.043004,0.000000,0.000000,0.000000,...,0.000000,0.121705,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.180618,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.180618,0.000000,0.180618


In [95]:
# 11.5 apply sklearn TfidfVectorizer
        # 11.5.1 adjectives
tr_idf_adjs_model  = TfidfVectorizer()
tf_idf_adjs_vector = tr_idf_adjs_model.fit_transform(corpus_adjs)

In [96]:
# 11.5.1 continued
adj_words_set = tr_idf_adjs_model.get_feature_names()

In [97]:
# 11.5.1 continued
tf_idf_adjs_array = tf_idf_adjs_vector.toarray()

In [98]:
# 11.5.1 continued
df_tf_adjs_idf = pd.DataFrame(tf_idf_adjs_array, columns = adj_words_set)

In [99]:
df_tf_adjs_idf

,authentic,beautiful,biological,clinical,different,early,evermore,finished,fundamental,great,...,several,simple,small,smaller,therapeutic,unquote,untouched,vast,vulnerable,wrong
0,0.000000,0.000000,0.00000,0.258490,0.516981,0.000000,0.000000,0.308432,0.000000,0.000000,...,0.308432,0.000000,0.000000,0.308432,0.308432,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.519708,0.000000,...,0.000000,0.435556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.249876,0.000000,0.00000,0.000000,0.249876,0.298154,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.215623,0.000000,0.000000,0.000000,0.000000,0.000000,0.499753,0.000000
4,0.223430,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.192802,0.000000,0.000000,0.000000,0.000000,0.000000,0.446860,0.000000
5,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.367489,0.000000,0.000000,0.307985,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.367489,0.000000,0.000000,0.000000,0.367489
6,0.000000,0.000000,0.34729,0.291056,0.000000,0.000000,0.000000,0.000000,0.000000,0.291056,...,0.000000,0.000000,0.502314,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.348676,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.292218,0.000000,0.000000,0.000000,0.000000,0.348676,0.348676,0.000000,0.000000


In [100]:
#11.5.1.1 Final shape
cols = df_tf_adjs_idf.columns
bt = df_tf_adjs_idf.apply(lambda x: x > 0)
final_shape_adjs = [bt.apply(lambda x: list(cols[x.values]), axis=1)]

In [101]:
print(final_shape_adjs) #might be better as dict

[0    [clinical, different, finished, hard, potentia...
1         [fundamental, ill, multidimensional, simple]
2                                    [replacementword]
3    [authentic, different, early, incredible, many...
4    [authentic, important, incredible, mental, muc...
5    [evermore, great, happy, nervous, old, sad, un...
6    [biological, clinical, great, imagine, much, o...
7    [beautiful, key, little, low, much, quiet, sim...
dtype: object]


In [102]:
# 11.5.2 adverbs
tr_idf_advs_model  = TfidfVectorizer()
tf_idf_advs_vector = tr_idf_advs_model.fit_transform(corpus_advs)

In [103]:
# 11.5.2 continued
adv_words_set = tr_idf_advs_model.get_feature_names()

In [104]:
# 11.5.2 continued
tf_idf_advs_array = tf_idf_advs_vector.toarray()

In [105]:
# 11.5.2 continued
df_tf_advs_idf = pd.DataFrame(tf_idf_advs_array, columns = adv_words_set)

In [106]:
df_tf_advs_idf

,absolutely,again,almost,alone,best,clearly,definitely,effectively,essentially,even,...,really,right,slightly,so,somewhat,sort,still,there,well,yet
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.429558,0.429558,0.000000,0.000000,0.000000,...,0.272375,0.360003,0.000000,0.000000,0.000000,0.000000,0.241098,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.337638,0.000000,0.000000,0.000000,0.000000,0.000000,0.337638,0.000000,...,0.000000,0.565934,0.337638,0.000000,0.337638,0.000000,0.379013,0.000000,0.282967,0.000000
4,0.388496,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.388496,0.000000,0.000000,...,0.492675,0.000000,0.000000,0.388496,0.000000,0.000000,0.218051,0.000000,0.325590,0.388496
5,0.000000,0.321076,0.000000,0.000000,0.321076,0.000000,0.000000,0.000000,0.000000,0.321076,...,0.407176,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.321076,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.263081,0.000000,0.000000,0.000000,0.000000,0.414902,0.232872,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.481403,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.270197,0.000000,0.000000,0.000000


In [107]:
#11.5.2.1 Final shape
cols = df_tf_advs_idf.columns
bt = df_tf_advs_idf.apply(lambda x: x > 0)
final_shape_advs = [bt.apply(lambda x: list(cols[x.values]), axis=1)]

In [108]:
print(final_shape_advs) #might be better as dict

[0    [clearly, definitely, however, real, really, r...
1                                               [kind]
2                                [mentally, overnight]
3    [almost, essentially, right, slightly, somewha...
4    [absolutely, effectively, really, so, still, w...
5    [again, best, even, kind, privately, quickly, ...
6      [further, kind, knowingly, really, sort, still]
7            [alone, fortunate, nearby, pretty, still]
dtype: object]
